In [1]:
import matplotlib.pyplot as plt
import numpy as np

from ase.build import bulk

from dask.distributed import Client
from dask_cuda import LocalCUDACluster

from abtem import FrozenPhonons, Probe, Potential

import os
os.environ["MKL_NUM_THREADS"] = "1"

from abtem import config
config.set({'device': 'gpu'})
config.set({'lazy': True})

Runtime: 9 s per frozen phonon on 1 NVidia RTX 2080 Ti

In [2]:
cluster = LocalCUDACluster(memory_limit='10gb', n_workers=1)
client = Client(cluster)

C:\Users\jacob\anaconda3\envs\abtem-dask-gpu\lib\site-packages\distributed\node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 61004 instead
  warnings.warn(
C:\Users\jacob\anaconda3\envs\abtem-dask-gpu\lib\site-packages\dask_cuda\utils.py:168: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  warnings.warn(


In [3]:
silicon = bulk('Si', crystalstructure='diamond', cubic=True)
silicon *= (74,74,100)
len(silicon)

4380800

In [4]:
frozen_phonons = FrozenPhonons(silicon, 4, {'Si':.1})

potential = Potential(frozen_phonons, gpts=2048, projection='infinite', slice_thickness=.5)

wave = Probe(energy=100e3, semiangle_cutoff=9.4)

In [5]:
measurement = wave.multislice(potential, (0,0)).diffraction_patterns().mean(0)

In [6]:
%%time
measurement.compute()

Wall time: 37.2 s
